In [1]:
## Name: Chandni Patel
## ID: A20455322
## CS 512 - Fall 2020

import cv2
import math
import numpy as np
import smoothing_convolution


In [2]:
class image_processor:
    
    #initialling the image processor
    def __init__(self, img):
        self.close = False
        self.curr_channel = 0
        self.TITLE = 'Image Processor'
        self.TRACKBAR = 'trackbar'
        self.support_keys = {ord('i'),ord('g'),ord('G'),ord('c')
                     ,ord('s'),ord('S'),ord('d'),ord('D'),ord('x')
                     ,ord('y'),ord('m'),ord('p'),ord('r'),ord('h')}
        print("\nPress h to display help.\n") 
        self.camera = False
        if (img == ''):
            self.camera = True
            self.capture = cv2.VideoCapture(0)
            valid, self.in_img = self.capture.read()
            if (valid == False):
                cv2.destroyAllWindows()
                self.capture.release()
            self.curr_img = np.copy(self.in_img) 
        else:
            self.in_img = cv2.imread(img)                    
            self.curr_img = np.copy(self.in_img)  
        self.get_preview()   

    #showing the image and waiting for key press
    def get_preview(self):
        while True:   
            if (self.close == True):
                self.exit()
                break            
            k = cv2.waitKey(10) & 0xFF
            if k == 27:
                self.exit()
                break
            if k == ord('w'):
                self.img_w()
            if k in self.support_keys:
                self.command_key(k)    
            cv2.imshow(self.TITLE, self.in_img)
            if (self.camera):
                if (self.set_image() == False):
                    self.exit()
                    break 
    
    #get image from current camera frame
    def set_image(self):
        valid, self.in_img = self.capture.read()
        if (valid == False):                
            return valid
        self.curr_img = np.copy(self.in_img) 

    #controlling all support keys
    def command_key(self, k):
        if k == ord('i'):
            self.img_i()        
        if k == ord('g'):
            self.img_g()
        if k == ord('G'):
            self.img_G()
        if k == ord('c'):
            self.curr_channel = (self.curr_channel+1)%3
            self.img_c()
        if k == ord('s'):
            self.img_s()
        if k == ord('S'):
            self.img_S()
        if k == ord('d'):
            self.img_d()
        if k == ord('D'):
            self.img_D()
        if k == ord('x'):
            self.img_x()
        if k == ord('y'):
            self.img_y()
        if k == ord('m'):
            self.img_m()
        if k == ord('p'):
            self.img_p()
        if k == ord('r'):
            self.img_r()                
        if k == ord('h'):
            self.img_h()

    #exit
    def exit(self):
        self.close = True
        cv2.destroyAllWindows()
        if (self.camera):
            self.capture.release()

    #tracker event not used due to recursive frame capturing
    def tracker(self, a):
        return
            
    #reload the original image
    def img_i(self):
        self.curr_img = np.copy(self.in_img)

    #save the current processed image
    def img_w(self):
        cv2.imwrite('out.jpg', self.curr_img)

    #convert the image to grayscale using openCV
    def img_g(self):
        self.curr_img = np.copy(self.in_img)  
        while True:   
            self.curr_img = cv2.cvtColor(self.in_img, cv2.COLOR_BGR2GRAY)

            k = cv2.waitKey(10) & 0xFF
            if k == 27:
                self.exit()
                break
            if k == ord('w'):
                self.img_w()
            if k in self.support_keys:
                break                  
            cv2.imshow(self.TITLE, self.curr_img)
            if (self.camera):
                if (self.set_image() == False):
                    self.exit()
                    break
        if k in self.support_keys:
                self.command_key(k)          
    
    #convert the image to grayscale using conversion function
    def img_G(self):
        self.curr_img = np.copy(self.in_img)  
        while True:   
            r = self.in_img[:,:,0]
            g = self.in_img[:,:,1]
            b = self.in_img[:,:,2]
            self.curr_img = ((0.299*r) + (0.587*g) + (0.114*b)).astype('uint8')
                  
            k = cv2.waitKey(10) & 0xFF
            if k == 27:
                self.exit()
                break
            if k == ord('w'):
                self.img_w()
            if k in self.support_keys:
                break
            cv2.imshow(self.TITLE, self.curr_img)
            if (self.camera):
                if (self.set_image() == False):
                    self.exit()
                    break
        if k in self.support_keys:
                self.command_key(k)                 

    #cycle through the color channels of the image showing a different channel every time the key is pressed
    def img_c(self):
        self.curr_img = np.copy(self.in_img)  
        while True:   
            in_img = self.in_img            
            temp_img = np.zeros((in_img.shape[0], in_img.shape[1], in_img.shape[2]), 'uint8')
            if self.curr_channel == 0:            
                temp_img[:, :, 2] = in_img[:, :, 2]            
            elif self.curr_channel == 1:
                temp_img[:, :, 1] = in_img[:, :, 1]
            else:
                temp_img[:, :, 0] = in_img[:, :, 0]
            self.curr_img = temp_img
                  
            k = cv2.waitKey(10) & 0xFF
            if k == 27:
                self.exit()
                break
            if k == ord('w'):
                self.img_w()
            if k in self.support_keys:
                break
            cv2.imshow(self.TITLE, self.curr_img)
            if (self.camera):
                if (self.set_image() == False):
                    self.exit()
                    break
        if k in self.support_keys:
                self.command_key(k)

    #convert the image to grayscale and smooth it using openCV    
    def img_s(self):
        self.gray_img = cv2.cvtColor(self.in_img, cv2.COLOR_BGR2GRAY)
        cv2.createTrackbar(self.TRACKBAR, self.TITLE, 1, 51, self.tracker)    
        self.curr_img = np.copy(self.gray_img)
        while True:   
            self.smoothing_trackbar()
            n = cv2.getTrackbarPos(self.TRACKBAR, self.TITLE)
                  
            k = cv2.waitKey(10) & 0xFF
            if k == 27:
                self.exit()
                break
            if k == ord('w'):
                self.img_w()
            if k in self.support_keys:
                break
            cv2.imshow(self.TITLE, self.curr_img)
            if (self.camera):
                if (self.set_image() == False):
                    self.exit()
                    break
        if k in self.support_keys:
                self.command_key(k)  

    #track bar for smoothing using openCV
    def smoothing_trackbar(self):
        n = cv2.getTrackbarPos(self.TRACKBAR, self.TITLE)
        if (n == 0):
            return
        if n%2 == 0:
            n -= 1
        self.gray_img = cv2.cvtColor(self.in_img, cv2.COLOR_BGR2GRAY)
        self.curr_img = cv2.GaussianBlur(self.gray_img,(n,n),0,0) 
                
    #convert the image to grayscale and smooth it using convolution
    def img_S(self):
        self.gray_img = cv2.cvtColor(self.in_img, cv2.COLOR_BGR2GRAY)
        cv2.createTrackbar(self.TRACKBAR, self.TITLE, 1, 51, self.tracker)   
        self.curr_img = np.copy(self.gray_img)
        while True:   
            self.smoothing_bar()
            cv2.imshow(self.TITLE, self.curr_img)

            k = cv2.waitKey(10) & 0xFF
            if k == 27:
                self.exit()
                break
            if k == ord('w'):
                self.img_w()
            if k in self.support_keys:
                break
            if (self.camera):
                if (self.set_image() == False):
                    self.exit()
                    break
        if k in self.support_keys:
                self.command_key(k)  

    #track bar for smoothing using convolution
    def smoothing_bar(self):
        n = cv2.getTrackbarPos(self.TRACKBAR, self.TITLE)
        if (n == 0):
            return
        if n%2 == 0:
            n -= 1
        self.gray_img = cv2.cvtColor(self.in_img, cv2.COLOR_BGR2GRAY)
        filter_x = np.ones((1, n))/n
        out_img = smoothing_convolution.apply_convolution(self.gray_img, filter_x)
        filter_y = np.ones((n, 1))/n
        self.curr_img = smoothing_convolution.apply_convolution(out_img, filter_y) 

    #downsample the image by a factor of 2 without smoothing
    def img_d(self):
        self.curr_img = np.copy(self.in_img)  
        while True:   
            self.curr_img = cv2.resize(self.in_img, (int(self.in_img.shape[1]/2), int(self.in_img.shape[0]/2)), interpolation=cv2.INTER_AREA)
            cv2.imshow(self.TITLE, self.curr_img)

            k = cv2.waitKey(10) & 0xFF
            if k == 27:
                self.exit()
                break
            if k == ord('w'):
                self.img_w()
            if k in self.support_keys:
                break
            if (self.camera):
                if (self.set_image() == False):
                    self.exit()
                    break
        if k in self.support_keys:
                self.command_key(k)  

    #downsample the image by a factor of 2 with smoothing
    def img_D(self):
        self.curr_img = np.copy(self.in_img)  
        while True:   
            self.curr_img = cv2.GaussianBlur(self.in_img,(51,51),0,0)
            self.curr_img = cv2.resize(self.curr_img, (int(self.in_img.shape[1]/2), int(self.in_img.shape[0]/2)), interpolation=cv2.INTER_AREA)
            cv2.imshow(self.TITLE, self.curr_img)

            k = cv2.waitKey(10) & 0xFF
            if k == 27:
                self.exit()
                break
            if k == ord('w'):
                self.img_w()
            if k in self.support_keys:
                break
            if (self.camera):
                if (self.set_image() == False):
                    self.exit()
                    break
        if k in self.support_keys:
                self.command_key(k)          

    #convert image to grayscale and perform convolution with an x derivative filter
    #normalize the obtained values to the range [0,255]
    def img_x(self):
        while True:    
            self.gray_img = cv2.cvtColor(self.in_img, cv2.COLOR_BGR2GRAY)
            k_x, k_y = cv2.getDerivKernels(1, 0, 3, True)
            grad_x = cv2.filter2D(self.gray_img, -1, k_y)
            grad_x = cv2.filter2D(grad_x, -1, k_x.T)
            self.curr_img = (np.float64(grad_x) * 255.0 / grad_x.max()).astype('uint8')
            cv2.imshow(self.TITLE, self.curr_img)

            k = cv2.waitKey(10) & 0xFF
            if k == 27:
                self.exit()
                break
            if k == ord('w'):
                self.img_w()
            if k in self.support_keys:
                break
            if (self.camera):
                if (self.set_image() == False):
                    self.exit()
                    break
        if k in self.support_keys:
                self.command_key(k)                

    #convert image to grayscale and perform convolution with an y derivative filter
    #normalize the obtained values to the range [0,255]
    def img_y(self):
        while True:   
            self.gray_img = cv2.cvtColor(self.in_img, cv2.COLOR_BGR2GRAY)
            k_x, k_y = cv2.getDerivKernels(0, 1, 3, True)
            grad_y = cv2.filter2D(self.gray_img, -1, k_x.T)
            grad_y = cv2.filter2D(grad_y, -1, k_y)
            self.curr_img = (np.float64(grad_y) * 255.0 / grad_y.max()).astype('uint8')
            cv2.imshow(self.TITLE, self.curr_img)

            k = cv2.waitKey(10) & 0xFF
            if k == 27:
                self.exit()
                break
            if k == ord('w'):
                self.img_w()
            if k in self.support_keys:
                break
            if (self.camera):
                if (self.set_image() == False):
                    self.exit()
                    break
        if k in self.support_keys:
                self.command_key(k)

    #show the magnitude of the gradient normalized to the range [0,255]
    #the gradient is computed based on the x and y derivative of the image
    def img_m(self):
        while True:   
            self.gray_img = cv2.cvtColor(self.in_img, cv2.COLOR_BGR2GRAY)
            k_x, k_y = cv2.getDerivKernels(1, 0, 3, True)
            grad_x = cv2.filter2D(self.gray_img, -1, k_y)
            grad_x = cv2.filter2D(grad_x, -1, k_x.T)
            grad_x = (np.float64(grad_x) * 255.0 / grad_x.max()).astype('uint8')
            k_x, k_y = cv2.getDerivKernels(0, 1, 3, True)
            grad_y = cv2.filter2D(self.gray_img, -1, k_x.T)
            grad_y = cv2.filter2D(grad_y, -1, k_y)
            grad_y = (np.float64(grad_y) * 255.0 / grad_y.max()).astype('uint8')
            grad_mag = np.hypot(grad_x, grad_y)
            grad_mag = np.float64(grad_mag)
            grad_mag *= 255.0 / grad_mag.max()
            self.curr_img = grad_mag.astype('uint8')
            cv2.imshow(self.TITLE, self.curr_img)

            k = cv2.waitKey(10) & 0xFF
            if k == 27:
                self.exit()
                break
            if k == ord('w'):
                self.img_w()
            if k in self.support_keys:
                break
            if (self.camera):
                if (self.set_image() == False):
                    self.exit()
                    break
        if k in self.support_keys:
                self.command_key(k)             
        
    #convert the image to grayscale and plot the gradient vectors of the image every N pixels
    #let the plotted gradient vectors have a length of K
    #plot the vectors as short line segments of length K    
    def img_p(self):
        self.gray_img = cv2.cvtColor(self.in_img, cv2.COLOR_BGR2GRAY)
        cv2.createTrackbar(self.TRACKBAR, self.TITLE, 50, 200, self.tracker)   
        self.curr_img = np.copy(self.gray_img)
        while True:   
            self.gradient_trackbar()
            cv2.imshow(self.TITLE, self.curr_img)

            k = cv2.waitKey(10) & 0xFF
            if k == 27:
                self.exit()
                break
            if k == ord('w'):
                self.img_w()
            if k in self.support_keys:
                break
            if (self.camera):
                if (self.set_image() == False):
                    self.exit()
                    break
        if k in self.support_keys:
                self.command_key(k)             
    
    #track bar to control N pixels for plotting gradient vectors
    def gradient_trackbar(self):
        n = cv2.getTrackbarPos(self.TRACKBAR, self.TITLE)
        self.gray_img = cv2.cvtColor(self.in_img, cv2.COLOR_BGR2GRAY)
        if n == 0: 
            return
        vec_len_K = 20
        self.curr_img = np.copy(self.gray_img)
        n_rows, n_cols = self.gray_img.shape[:2]
        grad_x, grad_y = np.gradient(self.gray_img) 
        row, col, x, y = 0, 0, 0, 0
        while row < n_rows: 
            col = 0
            while col < n_cols:
                y_grad = grad_y[row][col]
                x_grad = grad_x[row][col]
                if x_grad:
                    angle = math.atan(y_grad / x_grad)
                else:
                    angle = 1.5708 # 90 degrees        
                x = (math.cos(angle) * vec_len_K) + row
                y = (math.sin(angle) * vec_len_K) + col
                cv2.arrowedLine(self.curr_img, (col, row), (int(y), int(x)), (0, 0, 255), 1)
                col += n
            row += n

    #convert the image to grayscale and rotate it using an angle of theta degree
    #the rotation of the image should be performed using an inverse map so there are no holes in it    
    def img_r(self):
        self.gray_img = cv2.cvtColor(self.in_img, cv2.COLOR_BGR2GRAY)
        cv2.createTrackbar(self.TRACKBAR, self.TITLE, 0, 359, self.tracker)
        self.curr_img = np.copy(self.gray_img)
        while True:   
            self.rotation_trackbar()
            cv2.imshow(self.TITLE, self.curr_img)

            k = cv2.waitKey(10) & 0xFF
            if k == 27:
                self.exit()
                break
            if k == ord('w'):
                self.img_w()
            if k in self.support_keys:
                break
            if (self.camera):
                if (self.set_image() == False):
                    self.exit()
                    break
        if k in self.support_keys:
                self.command_key(k)             

    #track bar to control theta for rotation
    def rotation_trackbar(self):
        theta = cv2.getTrackbarPos(self.TRACKBAR, self.TITLE)
        self.gray_img = cv2.cvtColor(self.in_img, cv2.COLOR_BGR2GRAY)
        (r,c) = self.gray_img.shape[:2]
        (cr, cc) = (r//2, c//2)
        M = cv2.getRotationMatrix2D((cc, cr), -theta, 1.0)
        cos = np.abs(M[0, 0])
        sin = np.abs(M[0, 1])
        nc = int((r * sin) + (c * cos))
        nr = int((r * cos) + (c * sin))
        M[0, 2] += (nc / 2) - cc
        M[1, 2] += (nr / 2) - cr
        self.curr_img = cv2.warpAffine(self.gray_img, M, (nc, nr)) 
    
    #display a short description of the program, command line argument and support support_keys
    def img_h(self):
        print("\nSupport support_keys are as below:")
        print("i -> Reload the original image")
        print("w -> Save the image with current changes to the output file out.jpg")
        print("g -> Convert to grayscale using opencv")
        print("G -> Convert to grayscale without opencv")
        print("c -> Cycle through the RGB channels of the image")
        print("s -> Convert to grayscale and smooth using opencv (trackbar)")
        print("S -> Convert to grayscale and smooth using convolution (trackbar)")
        print("d -> Downsample the image by factor of 2")
        print("D -> Downsample the image by factor of 2 with smoothing")
        print("x -> Convert to grayscale and convolve with an x derivative filter. Normalize to [0, 255]")
        print("y -> Convert to grayscale and convolve with an y derivative filter. Normalize to [0, 255]")
        print("m -> Show the magnitude of the gradient normalized to [0, 255]")
        print("p -> Convert to grayscale and plot gradient vectors every N pixels (trackbar)")
        print("r -> Convert to grayscale and rotate with angle theta (trackbar)")
        print("h -> Display Help")
        print("ESC -> Exit\n")

In [3]:
ip = image_processor('in.jpg')


Press h to display help.

